In [16]:
import os
import sys
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from src.base_functions import turn_voltage
from src.superconductor_losses import thermal_incomes, cryo_surface,cryostat_losses

The following code snippets try to check the correctness of the transformer parameters, defined by doi.org/10.1088/0953-2048/20/5/010

The turn voltage calculated in 2 different ways:
   - from the core parameters
   - from the given line/phase voltages and turn number of the windings

The given parameters of the transformer core are:
 - the core diameter is 396 mm, the core radius is 198 mm
 - the flux density in the core is 1.275 T
 - the filling factor of the core was not given, we calculating by a usual 88 %
 - the network frequency is 50 Hz
 
Using the turn voltage base function of the library:

In [2]:
ut = turn_voltage(1.275, 198.0, 0.88, 50.0)
print('turn voltage:',round(ut,2), 'V')

turn voltage: 30.68 V


Turn voltage can be calculated from the winding parameters of the star/star connected three phase transformer:

In [3]:
Np = 262.0 # turns in the primary winding
Up = 10500 # V 
utp = Up/Np/1.73
print("Turn voltage from primary winding data: ", round(utp,2), "V")

# secondary winding
Ns = 10
Us = 400
uts= Us/1.73/Ns
print("Turn voltage from secondary winding data:",round(uts,2), "V")

Turn voltage from primary winding data:  23.17 V
Turn voltage from secondary winding data: 23.12 V


Therefore, some parameter of the core is wrong, maybe the flux density of the core is a bit lower in the practice, in this case the flux density should be 0.963 T

In [30]:
ut = turn_voltage(1.275, 170.0, 0.89, 50.0)
print(ut)

22.87


In [5]:
from importlib_resources import files
from src.two_winding_model import TransformerDesign, TwoWindingModel

path = files("data").joinpath("630kVA_sc_transformer.json")

import json

with open(path) as json_file:
    data = json.load(json_file)

transformer = TransformerDesign.from_dict(data)

trafo_model = TwoWindingModel(input=transformer)
trafo_model.calculate(is_sc=True)

# FEM calculation
trafo_model.fem_simulation(detailed_output = False)

Mesh Generator: Initial mesh generation
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:01.214 s
Magnetic Energy 10.102712946381097
zb, ib: 0.25 ohm 909.33 A
SCI: 3.02 [%]
Bax  [HV] = 24.18 [mT]
Brad [HV] = 12.7 [mT]
Bax  [LV] = 11.91 [mT]
Brad [Lv] = 13.69 [mT]


The following command can list the main results of the calculation:

In [6]:
print(trafo_model.hv_winding)

WindingDesign(inner_radius=288.5, thickness=10.2, winding_height=354.83, filling_factor=50.0, current_density=3.78, mass=64.1, dc_loss=0.0, ac_loss=10349.737315374277, outer_radius=298.7, cable_length=7880.11, amper_turns=6840.4)


In [7]:
print(trafo_model.lv_winding)

WindingDesign(inner_radius=244.0, thickness=6.0, winding_height=342.5, filling_factor=50.0, current_density=6.62, mass=30.6, dc_loss=0.0, ac_loss=4940.748521654515, outer_radius=250.0, cable_length=3761.8, amper_turns=6802.1)


In [25]:
trafo_model.fem_simulation(detailed_output = False)
print('analytical sci:', trafo_model.results.sci)

Magnetic EnergyMesh Generator: Initial mesh generation
 10.102712943262212
zb, ib: 0.25 ohm 909.33 A
SCI: 3.02 [%]
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:01.202 s
Bax  [HV] = 24.18 [mT]
Brad [HV] = 12.7 [mT]
Bax  [LV] = 11.91 [mT]
Brad [Lv] = 13.69 [mT]
analytical sci: 1.67


The resulting fem-based short circuit impedance is 2.28 %, which is very close to the given 2.45 %, however, the analytically calculated sci is only 1.7%. The reason is that the analytical formula contains the turn voltage, which is different, the core somehow not sized properly. 

In [26]:
from src.superconductor_losses import cryo_surface,cryostat_losses, magnusson_ac_loss, supra_winding_ac_loss
a_cs = cryo_surface(205.0, 370, 800)  # [mm2]
cryo_loss = cryostat_losses(a_cs)
#thermal_loss = thermal_incomes(self.input.required.lv.ph_current, self.input.required.hv.ph_current)

print('a_cs:',a_cs)
#print('cryostat losses:', cryo_loss)
print(cryo_loss)

# HV winding
print('hv:',magnusson_ac_loss(24.2*1e-3, 12.7*1e-3, 50, 34.5, Ic=115), 'W/m')
print('lv:',magnusson_ac_loss(11.9*1e-3, 13.7*1e-3, 50, 902/20, Ic=115), ',W/m')

# Calculation with the averaged values of Bax and Brad along the winding
B_hv =  [(0.011433322839513704, 0.012699025485460425), (0.018933806949452736, 0.005230937068120056), (0.020965518246982395, 0.0030748333510217662), (0.022150364633813535, 0.0020912742032237475), (0.02290724530387911, 0.0015658509766911521), (0.02339652266536152, 0.0012678793822082923), (0.023725376383052166, 0.00109892273180634), (0.023935062181081213, 0.0010086370273540979), (0.024068554631890116, 0.0009674387479634561), (0.024149590760132067, 0.000958728315879398), (0.02417897594277435, 0.0009844094746398468), (0.02417202624967385, 0.0010112760123917042), (0.024127634300657175, 0.0010374720680452754), (0.02402747475883754, 0.0010415087262813927), (0.023868892182893786, 0.001022807939854431), (0.023623027309799538, 0.0009612379640060453), (0.02321806402269372, 0.0008331465044477682), (0.022639879555103826, 0.000608199029066725), (0.021680382237800555, 0.00042832155749295), (0.020217882565412737, 0.0016489134058446098), (0.01783499675174277, 0.004253943059708831)]
print('hv average:',supra_winding_ac_loss(B_hv, 50, 69, Ic=170, kappa=2.0), 'W/m')

B_lv = [(0.011433322839513704, 0.012699025485460425), (0.018933806949452736, 0.005230937068120056), (0.020965518246982395, 0.0030748333510217662), (0.022150364633813535, 0.0020912742032237475), (0.02290724530387911, 0.0015658509766911521), (0.02339652266536152, 0.0012678793822082923), (0.023725376383052166, 0.00109892273180634), (0.023935062181081213, 0.0010086370273540979), (0.024068554631890116, 0.0009674387479634561), (0.024149590760132067, 0.000958728315879398), (0.02417897594277435, 0.0009844094746398468), (0.02417202624967385, 0.0010112760123917042), (0.024127634300657175, 0.0010374720680452754), (0.02402747475883754, 0.0010415087262813927), (0.023868892182893786, 0.001022807939854431), (0.023623027309799538, 0.0009612379640060453), (0.02321806402269372, 0.0008331465044477682), (0.022639879555103826, 0.000608199029066725), (0.021680382237800555, 0.00042832155749295), (0.020217882565412737, 0.0016489134058446098), (0.01783499675174277, 0.004253943059708831)]
print('lv average:',supra_winding_ac_loss(B_lv, 50, 69, Ic=170, kappa=1.5), 'W/m')


a_cs: 3188323.844311941
29.08
hv: 0.05106904243619691 W/m
lv: 0.05488619049790877 ,W/m
hv average: 0.018 W/m
lv average: 0.018 W/m


In [27]:
print("Values without average:")
r_m = 244
length = Np * 2*r_m*3.14 * 1e-3 
print(length*3)

length_sec = 10*22*2*3.14*1e-3*290
loss_sec = 0.744 *length_sec*3
print(length_sec*3, loss_sec)

print("Loss calculation for 21 segments in the hv winding:")
print('P_sec', 0.051*length_sec*3 )
print('P_pri', 0.055*length*3)



Values without average:
1204.40352
1201.9920000000002 894.282048
Loss calculation for 21 segments in the hv winding:
P_sec 61.301592
P_pri 66.24219360000001


In [29]:
P_ac_max = length*0.051*3 + length_sec*0.055*3
P_ac_avg = length*0.018*3 + length_sec*0.018*3

print('P_ac (max) = ', round(P_ac_max))
print('P_ac (avg) = ',round(P_ac_avg))

P_ac (max) =  128
P_ac (avg) =  43


In [19]:
pt = thermal_incomes(34.5,909)
print('Thermal losses:', pt, 'W')

Thermal losses: 254.75 W
